In [1]:
import numpy as np
import string
import json
import random
import nltk
from collections import defaultdict
import pickle
from nltk.corpus import stopwords

In [2]:
# Load data
print 'Reading data...'
data = json.load(open('winemag-data-130k-v2.json'))
print 'done\n'
print 'Sample data:'
print(data[0])
print data[0]['province']
print('Length of the dataset is ' + str(len(data)))

Reading data...
done

Sample data:
{u'province': u'Sicily & Sardinia', u'region_2': None, u'description': u"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", u'designation': u'Vulk\xe0 Bianco', u'title': u'Nicosia 2013 Vulk\xe0 Bianco  (Etna)', u'region_1': u'Etna', u'country': u'Italy', u'taster_name': u'Kerin O\u2019Keefe', u'variety': u'White Blend', u'points': u'87', u'winery': u'Nicosia', u'price': None, u'taster_twitter_handle': u'@kerinokeefe'}
Sicily & Sardinia
Length of the dataset is 129971


In [5]:
# shuffle and split dataset
random.shuffle(data)
# print(data[0])
train = data[:120000]
test = data[120000:]
print( str(len(train)) + ' samples for taining and ' + str(len(test)) + ' samples for testing\n')

120000 samples for taining and 9971 samples for testing



In [ ]:
print 'Creating features...'
# extract feature for training set and testing set
punct = set(string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()
# sw = nltk.corpus.stopwords.words('english')
# print sw
# get necessary overall information
# one-hot encoded info
variety_all = []
winery_all = []
regions1_all = []
word_all = []
wordCount = defaultdict(int) # # of appearance of w in all documents
docCount = defaultdict(int) # # of docs of all docs that contains word w
tf = []

desig_word_all = []
desigWordCount = defaultdict(int)
desigDocCount = defaultdict(int)
desigTf = []
for l in train:
    if l['variety'] not in variety_all:
        variety_all.append(l['variety'])
    if l['winery'] not in winery_all:
        winery_all.append(l['winery'])
    if l['region_1'] not in regions1_all:
        regions1_all.append(l['region_1'])

    wordCountTmp = defaultdict(int)
    for w in l['description'].split():
        w = ''.join([c for c in w.lower() if not c in punct])
        if w not in stopwords.words('english'):
            w = stemmer.stem(w)
            if w not in word_all:
                word_all.append(w)
            wordCount[w] += 1
            wordCountTmp[w] += 1
            if w not in docCount.keys():
                docCount[w] += 1
    tf.append(wordCountTmp)

Creating features...


In [ ]:
print('number of words in training' + str(len(word_all)))

In [ ]:
# Create features
feat_descrip = []
feat_desig = []
feat_points = []
feat_price = []
feat_variety = []
feat_winery = []
idx = 0
for l in train:
    tmpDescrip = np.zeros(shape=(len(word_all)))
    for w in l['description'].split():
        w = ''.join([c for c in w.lower() if not c in punct])
        if w not in stopwords.words('english'):
            w = stemmer.stem(w)
            wordIdx = word_all.index(w)
            tmpDescrip[wordIdx] = tf[idx][w] * np.log(120000/docCount[w])
    feat_descrip.append(tmpDescrip)
    # tmpDesig = np.zeros(shape=(len(desig_word_all)))
    # for w in l['designation'].split():
    #     w = ''.join([c for c in w.lower() if not c in punct])
    #     w = stemmer.stem(w)
    #     wordIdx = desig_word_all.index(w)
    #     tmpDesig[wordIdx] = tf[idx][w] * np.log(120000/docCOunt[w])
    # feat_desig.append(tmpDesig)
    feat_points.append(l['points'])
    feat_price.append(l['price'])
    tmpVariety = np.zeros(shape=(len(variety_all)))
    feat_variety.append(tmpVariety[variety_all.index(l['variety'])])
    tmpWinery = np.zeros(shape=(len(winery_all)))
    feat_winery.append(tmpWinery[winery_all.index(l['winery'])])
    idx += 1

In [ ]:
# testing
variety_all = []
winery_all = []
regions1_all = []
word_all = []
wordCount = defaultdict(int) # # of appearance of w in all documents
docCount = defaultdict(int) # # of docs of all docs that contains word w
tf = []

print 'Creating features...'
# desig_word_all = []
# desigWordCount = defaultdict(int)
# desigDocCount = defaultdict(int)
# desigTf = []
for l in test:
    if l['variety'] not in variety_all:
        variety_all.append(l['variety'])
    if l['winery'] not in winery_all:
        winery_all.append(l['winery'])
    if l['region_1'] not in regions1_all:
        regions1_all.append(l['region_1'])

    wordCountTmp = defaultdict(int)
    for w in l['description'].split():
        w = ''.join([c for c in w.lower() if not c in punct])
        if w not in stopwords.words('english'):
            w = stemmer.stem(w)
            if w not in word_all:
                word_all.append(w)
            wordCount[w] += 1
            wordCountTmp[w] += 1
            if w not in docCount.keys():
                docCount[w] += 1
    tf.append(wordCountTmp)

print('number of words in training' + str(len(word_all)))

# Create features
feat_descrip_test = []
# feat_desig = []
feat_points_test = []
feat_price_test = []
feat_variety_test = []
feat_winery_test = []
idx = 0
for l in test:
    tmpDescrip = np.zeros(shape=(len(word_all)))
    for w in l['description'].split():
        w = ''.join([c for c in w.lower() if not c in punct])
        if w not in stopwords.words('english'):
            w = stemmer.stem(w)
            wordIdx = word_all.index(w)
            tmpDescrip[wordIdx] = tf[idx][w] * np.log(9971/docCount[w])
    feat_descrip_test.append(tmpDescrip)
    # tmpDesig = np.zeros(shape=(len(desig_word_all)))
    # for w in l['designation']:
    #     w = ''.join([c for c in w.lower() if not c in punct])
    #     w = stemmer.stem(w)
    #     wordIdx = desig_word_all.index(w)
    #     tmpDesig[wordIdx] = tf[idx][w] * np.log(9971/docCOunt[w])
    # feat_desig.append(tmpDesig)
    feat_points_test.append(l['points'])
    feat_price_test.append(l['price'])
    tmpVariety = np.zeros(shape=(len(variety_all)))
    feat_variety_test.append(tmpVariety[variety_all.index(l['variety'])])
    tmpWinery = np.zeros(shape=(len(winery_all)))
    feat_winery_test.append(tmpWinery[winery_all.index(l['winery'])])
    idx += 1    